In [ ]:
#pip install lyricsgenius
import lyricsgenius as lg
#api_key is your TOKEN from Genius Developers. It will look like that.
api_key = "AVD4yMSZU3BsiGEpnohBIbnPj0EweXieJIgCVsaxYxrbxQAqWrN6nCaOQdsH9f3B"
genius = lg.Genius(api_key)
genius = lg.Genius(api_key, skip_non_songs=True, remove_section_headers=True)
#genius.verbose = False
genius.timeout = 120

In [ ]:
import pandas
data = pandas.read_excel("dataset.xlsx")
print(data[:5])

songs = data['song'].tolist()
print(songs[0:10])

artists = data['artist'].tolist()
print(artists[0:10])

#peak_rank = data['peak.rank'].tolist()
#print(peak_rank[0:10])

In [ ]:
#remove whatever is after Featuring, &, With, X and x, Genius has problems with that
for i in range (0, len(artists)):
    if "Featuring" in str(artists[i]):
        artists[i] = artists[i].split("Featuring", 1)[0]
        
for i in range (0, len(artists)):
    if "&" in str(artists[i]):
        artists[i] = artists[i].split("&", 1)[0] 
        
for i in range (0, len(artists)):
    if "With" in str(artists[i]):
        artists[i] = artists[i].split("With", 1)[0]

for i in range (0, len(artists)):
    if "X " in str(artists[i]):
        artists[i] = artists[i].split("X ", 1)[0]

for i in range (0, len(artists)):
    if "x" in str(artists[i]):
        artists[i] = artists[i].split("X ", 1)[0]
        
import re        
for i in range (0, len(songs)): #delete anything what's in the brackets in title of the song
    if "(" in songs[i]:
        songs[i] = re.sub("[\(\[].*?[\)\]]", "", songs[i])

In [ ]:
import string

last_words = [[] for x in range(0, len(songs))]

for i in range (0, len(songs)):
    song = genius.search_song(songs[i], artists[i])
    try:
        lyrics = song.lyrics
        lyrics = str(lyrics)
        lyrics = re.sub("[\(\[].*?[\)\]]", "", lyrics) #deleting everything what is in brackets as those are usually vocalizations or repeated previous word/phrase
        lyrics = lyrics.replace("yeah", "").replace("woah", "").replace("ayy", "").replace("hah", "").replace("ooh", "").replace("uh-oh", "").replace("oohooh", "")
        lyrics = lyrics.translate(str.maketrans('', '', string.punctuation))
        lyrics_list = lyrics.split("\n")
        lyrics_list.pop(0)
        while '' in lyrics_list:
            lyrics_list.remove('')
        while ' ' in lyrics_list:
            lyrics_list.remove(' ')
        for el in lyrics_list:
            verse = el.split(' ')
            while '' in verse:
                verse.remove('')
            if len(verse) != 0: 
                last_words[i].append(verse[-1])
        if(len(last_words[i])) > 300 or len(last_words[i]) == 0: #sometimes the program finds non-lyrics but different data about e.g. songs from The Voice, then the length of that is usually longer than 300 and none of the normal songs have 300 lines of lyrics, therefore it can be assumed that if the len>300, it's not lyrics
            last_words[i] = ["no lyrics found"]
        for j in range(0, len(last_words[i])):
            if "Embed" in last_words[i][j]:
                last_words[i][j] = last_words[i][j].replace("Embed", "").rstrip("1234567890") #deleting Embed part of the word
        while '' in last_words[i]:
            last_words[i].remove('')
    except AttributeError: #sometimes the code can't find the lyrics, then the information is added to the list
        last_words[i].append("no lyrics found")
        pass

print("Done")

In [ ]:
#indices of the songs that lyrics haven't been found, need to delete those songs
no_lyrics = [index for index in range(len(last_words)) if last_words[index] == ['no lyrics found']]
print(no_lyrics)
print(len(no_lyrics))

In [ ]:
import pandas as pd

df = pd.DataFrame(last_words)
writer = pd.ExcelWriter('out.xlsx', engine='xlsxwriter')
df.to_excel(writer, index=False)
writer.save()
print("Done")

In [ ]:
indices_no_lyrics = pd.DataFrame(no_lyrics)
writer = pd.ExcelWriter('indices_no_lyrics.xlsx', engine='xlsxwriter')
indices_no_lyrics.to_excel(writer, index=False)
writer.save()
print("Done")